In [41]:
import csv
import os
import time
import numpy as np
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
import mindspore
from mindspore import nn
from mindspore import context
from mindspore import dataset
from mindspore.train.callback import TimeMonitor, LossMonitor
from mindspore import Tensor
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

# 设置MindSpore的执行模式和设备
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")


# 定义一些参数
cfg = edict({
    'data_size': 150, # 数据集大小
    'train_size': 120, # 训练集大小
    'test_size': 30, # 测试集大小
    'feature_number': 4, # 输入特征数
    'num_class': 3, # 分类类别
    'batch_size': 30,
    'data_dir': 'iris.data', # 数据集所在路径
    'epoch_size': 20, # 训练周期
    'learning_rate': 0.05 # 学习率
})


# 读取数据并进行预处理:
with open(cfg.data_dir) as csv_file:
    data = list(csv.reader(csv_file, delimiter=','))

# 共150条数据，将数据集的4个属性作为自变量X。将数据集的3个类别映射为{0, 1，2}，作为因变量Y。
label_map = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
X = np.array([[float(x) for x in s[:-1]] for s in data[:cfg.data_size]], np.float32)
Y = np.array([label_map[s[-1]] for s in data[:150]], np.int32)

# 将数据集分为训练集120条，测试集30条。
train_idx = np.random.choice(cfg.data_size, cfg.train_size, replace=False)
test_idx = np.array(list(set(range(cfg.data_size)) - set(train_idx)))
X_train, Y_train = X[train_idx], Y[train_idx]
X_test, Y_test = X[test_idx], Y[test_idx]


# 将读取的数据转换为dataset
def generate_dataset():
    # 训练集
    XY_data_train = list(zip(X_train, Y_train))
    train_dataset = dataset.GeneratorDataset(XY_data_train, ['x', 'y'])
    train_dataset = train_dataset.shuffle(buffer_size=cfg.train_size).batch(cfg.batch_size, drop_remainder=True)
    #测试集
    XY_data_test = list(zip(X_test, Y_test))
    test_dataset = dataset.GeneratorDataset(XY_data_test, ['x', 'y'])
    test_dataset = test_dataset.shuffle(buffer_size=cfg.test_size).batch(cfg.test_size, drop_remainder=True)
    
    return train_dataset, test_dataset


# 训练
def train(network, optimizer, train_dataset):
    loss_function = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    model = Model(network, loss_fn=loss_function, optimizer=optimizer, metrics={"acc"})
    model.train(cfg.epoch_size, train_dataset)
    
    return model


# 测试
def test(model, de_test):
    # 计算输出总体准确率
    accuracy = model.eval(de_test, dataset_sink_mode=False).get("acc")
    print(f"预测准确率: {accuracy}\n")
    

# 定义创建网络和优化器的函数
def create_network_and_optimizer(optimizer_type):
    # 创建全连接网络
    network = nn.Dense(cfg.feature_number, cfg.num_class)
    # 根据优化器类型创建优化器
    if optimizer_type == 'SGD':
        optimizer = nn.SGD(network.trainable_params(), cfg.learning_rate)
    elif optimizer_type == 'Momentum':
        optimizer = nn.Momentum(network.trainable_params(), cfg.learning_rate, 0.9)
    elif optimizer_type == 'Adam':
        optimizer = nn.Adam(network.trainable_params(), learning_rate=cfg.learning_rate)
    else:
        optimizer = None  # 无优化器

    return network, optimizer


# 对于各种优化器，进行训练和测试
for optimizer_type in ['无优化器', 'SGD', 'Momentum', 'Adam']:
    print(f'{optimizer_type}：')
    # 生成数据集
    de_train, de_test = generate_dataset()
    # 创建网络和优化器
    network, optimizer = create_network_and_optimizer(optimizer_type)
    # 进行训练
    model = train(network, optimizer, de_train)
    # 进行测试
    test(model, de_test)



无优化器：


[ERROR] ANALYZER(3821,7f212b77f740,python):2023-05-31-17:58:44.500.813 [mindspore/ccsrc/pipeline/jit/static_analysis/async_eval_result.cc:66] HandleException] Exception happened, check the information as below.

The function call stack (See file '/home/ma-user/work/rank_0/om/analyze_fail.dat' for more details):
# 0 In file /home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/mindspore/nn/wrap/cell_wrapper.py(112)
        return self._loss_fn(out, label)
               ^
# 1 In file /home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/mindspore/nn/loss/loss.py(313)
        x = F.square(logits - labels)
                     ^



ValueError: For 'Sub', x.shape and y.shape are supposed to broadcast, where broadcast means that x.shape[i] = 1 or -1 or y.shape[i] = 1 or -1 or x.shape[i] = y.shape[i], but now x.shape and y.shape can not broadcast, got i: -1, x.shape: [30, 3], y.shape: [30].

In [26]:
%tb


AttributeError: 'EasyDict' object has no attribute 'epoch_size'